In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 99)

In [27]:
airports = pd.read_csv('airports.csv')

In [3]:
#clean the flight data
flights = pd.read_csv('flights.csv')
AAflights = flights[flights['AIRLINE'] == 'AA']#filter for just flights of the airline we are hired by
AAflights.reset_index(inplace=True, drop=True)
AAflights['AIR_SYSTEM_DELAY'] = AAflights['AIR_SYSTEM_DELAY'].fillna(0)
AAflights['SECURITY_DELAY'] = AAflights['SECURITY_DELAY'].fillna(0)
AAflights['AIRLINE_DELAY'] = AAflights['AIRLINE_DELAY'].fillna(0)
AAflights['LATE_AIRCRAFT_DELAY'] = AAflights['LATE_AIRCRAFT_DELAY'].fillna(0)
AAflights['WEATHER_DELAY'] = AAflights['WEATHER_DELAY'].fillna(0)
AAflights.drop(axis=1, labels=['YEAR','MONTH','DAY','DAY_OF_WEEK','AIRLINE','FLIGHT_NUMBER'],inplace=True)

C:\Users\frype\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\frype\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\frype\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\frype\Anaconda3\lib\sit

In [28]:
#clean the plane data
planes = pd.read_csv('planes.csv')
AAfleet = planes
fleetNumbers = AAflights['TAIL_NUMBER'].unique()
removeList = []
for planeIndex in range(0,len(planes)):
    if AAfleet.loc[planeIndex]['TAIL_NUMBER'] not in fleetNumbers:
        #remove from fleet
        removeList.append(planeIndex)
AAfleet.drop(removeList,axis=0,inplace = True)
AAfleet.reset_index(drop=True, inplace = True)

In [29]:
AAfleet

,TAIL_NUMBER,PASSENGER_CAPACITY,FUEL_COST/MILE,BASE_COST/FLIGHT
0,N3KUAA,524,38.30,10384.99
1,N171US,368,33.60,7150.24
2,N3HYAA,5,0.27,463.34
3,N584UW,85,22.20,2409.66
4,N3LAAA,85,21.70,2373.45
...,...,...,...,...
1039,N7LCAA,524,35.30,9179.13
1040,N885AA,368,31.60,7040.58
1041,N7LLAA,85,23.50,2264.60
1042,N7LUAA,85,21.20,2286.18


### Information to gather for each plane in the fleet:
- Number of flights
- total miles traveled
- max passangers carried over all flights
- total cost of:
    - gate rental
    - fuel
    - net delay fees

In [30]:
#get number of flights for each plane in the fleet
flightsGrouped = AAflights.groupby('TAIL_NUMBER')
flightCount = flightsGrouped['ORIGIN_AIRPORT'].count()-flightsGrouped['CANCELLATION_REASON'].count()
flightCount = flightCount.to_frame()
flightCount.reset_index(inplace=True)
flightCount.columns = 'TAIL_NUMBER TOTAL_FLIGHTS'.split()
AAfleet = pd.merge(AAfleet,flightCount,on='TAIL_NUMBER')

In [31]:
#calculate and add column for the total number of passangers carried if each is at full capacity
AAfleet['TOTAL_MAX_PASSANGERS'] = AAfleet['PASSENGER_CAPACITY'] * AAfleet['TOTAL_FLIGHTS']

In [32]:
#get total distance traveled by each plane
planeDistance = AAflights[AAflights['CANCELLED'] == 0].groupby('TAIL_NUMBER')['DISTANCE'].sum()
planeDistance = planeDistance.to_frame()
planeDistance.reset_index(inplace=True)
planeDistance.columns = 'TAIL_NUMBER MILES_TRAVELED'.split()
AAfleet = pd.merge(AAfleet,planeDistance,on='TAIL_NUMBER')

In [33]:
#calculate the fuel cost for each plane
AAfleet['TOTAL_FUEL_COST'] = AAfleet['FUEL_COST/MILE'] * AAfleet['MILES_TRAVELED']
AAfleet

,TAIL_NUMBER,PASSENGER_CAPACITY,FUEL_COST/MILE,BASE_COST/FLIGHT,TOTAL_FLIGHTS,TOTAL_MAX_PASSANGERS,MILES_TRAVELED,TOTAL_FUEL_COST
0,N3KUAA,524,38.30,10384.99,1048,549152,1169348,44786028.40
1,N171US,368,33.60,7150.24,660,242880,732412,24609043.20
2,N3HYAA,5,0.27,463.34,732,3660,860026,232207.02
3,N584UW,85,22.20,2409.66,679,57715,797016,17693755.20
4,N3LAAA,85,21.70,2373.45,1011,85935,1166609,25315415.30
...,...,...,...,...,...,...,...,...
1039,N7LCAA,524,35.30,9179.13,2,1048,4684,165345.20
1040,N885AA,368,31.60,7040.58,20,7360,27073,855506.80
1041,N7LLAA,85,23.50,2264.60,3,255,7026,165111.00
1042,N7LUAA,85,21.20,2286.18,2,170,4684,99300.80


In [34]:
#pull fee pricing from the airport listings
flightFees = airports
flightFees.drop(axis=1, labels=['AIRPORT','CITY','STATE','COUNTRY','LATITUDE','LONGITUDE'],inplace=True)
flightFees.columns = ['ORIGIN_AIRPORT','BASE_GATE_USE_COST','AIRLINE_DELAY_FEE/MIN','AIR_SYSTEM_DELAY_FEE/MIN','SECURITY_DELAY_FEE/MIN']
flightsWFeePrices = pd.merge(AAflights,flightFees,on='ORIGIN_AIRPORT')
flightsWFeePrices = flightsWFeePrices[flightsWFeePrices['CANCELLED']==0]

In [35]:
#calculate fee costs on a per flight basis
flightsWFeePrices['AIR_SYSTEM_DELAY_FEE'] = flightsWFeePrices['AIR_SYSTEM_DELAY_FEE/MIN']*flightsWFeePrices['AIR_SYSTEM_DELAY']*-1
flightsWFeePrices['SECURITY_DELAY_FEE'] = flightsWFeePrices['SECURITY_DELAY_FEE/MIN']*flightsWFeePrices['SECURITY_DELAY']*-1
flightsWFeePrices['AIRLINE_DELAY_FEE'] = flightsWFeePrices['AIRLINE_DELAY_FEE/MIN']*flightsWFeePrices['AIRLINE_DELAY']
flightsWFeePrices['TOTAL_FEES'] = flightsWFeePrices['AIR_SYSTEM_DELAY_FEE']+flightsWFeePrices['SECURITY_DELAY_FEE']+flightsWFeePrices['AIRLINE_DELAY_FEE']

In [36]:
#clean out un-needed information from our flight list
dropList=['ORIGIN_AIRPORT','DESTINATION_AIRPORT','SCHEDULED_DEPARTURE','DEPARTURE_TIME','DEPARTURE_DELAY','TAXI_OUT','WHEELS_OFF','SCHEDULED_TIME','AIR_TIME','DISTANCE','WHEELS_ON','TAXI_IN','SCHEDULED_ARRIVAL','ARRIVAL_TIME','ARRIVAL_DELAY','DIVERTED','CANCELLED','CANCELLATION_REASON','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY','AIRLINE_DELAY_FEE/MIN','AIR_SYSTEM_DELAY_FEE/MIN','SECURITY_DELAY_FEE/MIN']
flightsWFeePrices.drop(axis=1,labels=dropList,inplace=True)

In [37]:
#calculate plane fees and gate costs
planeFees = flightsWFeePrices.groupby('TAIL_NUMBER')['TOTAL_FEES'].sum()
gateUse = flightsWFeePrices.groupby('TAIL_NUMBER')['BASE_GATE_USE_COST'].sum()

#add plane fees
planeFees = planeFees.to_frame()
planeFees.reset_index(inplace=True)
planeFees.columns = 'TAIL_NUMBER TOTAL_FEES'.split()
AAfleet = pd.merge(AAfleet,planeFees,on='TAIL_NUMBER')

#add gate use cost
gateUse = gateUse.to_frame()
gateUse.reset_index(inplace=True)
gateUse.columns = 'TAIL_NUMBER TOTAL_GATE_COST'.split()
AAfleet = pd.merge(AAfleet,gateUse,on='TAIL_NUMBER')

In [38]:
#find total costs and costs/passanger
AAfleet['TOTAL_COST']=AAfleet['TOTAL_FUEL_COST']+AAfleet['TOTAL_FEES']+AAfleet['TOTAL_GATE_COST']
AAfleet['COST/PASSANGER'] = AAfleet['TOTAL_COST'] / AAfleet['TOTAL_MAX_PASSANGERS']

In [39]:
#sort out the values based on the costs/passanger and output to a new CSV
AAfleet.sort_values('COST/PASSANGER',inplace=True)
AAfleet.reset_index(drop=True, inplace=True)
AAfleet.to_csv('AAfleetBaseAnalysis.csv',index=False)

In [40]:
AAfleet

,TAIL_NUMBER,PASSENGER_CAPACITY,FUEL_COST/MILE,BASE_COST/FLIGHT,TOTAL_FLIGHTS,TOTAL_MAX_PASSANGERS,MILES_TRAVELED,TOTAL_FUEL_COST,TOTAL_FEES,TOTAL_GATE_COST,TOTAL_COST,COST/PASSANGER
0,N961UW,524,36.0,9282.83,931,487844,376456,13552416.0,-24344.25,2935913.01,16463984.76,33.748462
1,N963UW,368,31.2,6904.66,1011,372048,324912,10137254.4,-4129.62,2979643.24,13112768.02,35.244829
2,N947UW,368,33.1,7691.30,989,363952,360876,11944995.6,8534.59,2933604.34,14887134.53,40.904115
3,N712US,524,35.8,9651.25,751,393524,416602,14914351.6,21849.23,2112786.24,17048987.07,43.323881
4,N952UW,368,31.8,7406.90,868,319424,356350,11331930.0,6615.57,2553245.17,13891790.74,43.490128
...,...,...,...,...,...,...,...,...,...,...,...,...
1039,N5ECAA,21,15.7,3358.02,773,16233,1390348,21828463.6,119055.48,2748366.22,24695885.30,1521.338342
1040,N7BKAA,21,16.8,3379.26,58,1218,100325,1685460.0,5411.48,182828.81,1873700.29,1538.341782
1041,N373AA,21,16.8,3525.35,237,4977,410732,6900297.6,46965.51,874945.47,7822208.58,1571.671404
1042,N7LDAA,21,15.4,3264.34,3,63,5805,89397.0,189.95,11012.79,100599.74,1596.821270
